In [ ]:
import networkx    #all the packaes I will import I import here
import urllib2
import re
import matplotlib
from nltk.tokenize import word_tokenize
import nltk
import itertools
data = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')
homer=data.read()




In [ ]:
def read_nodes(gfile):
    
    tokens = nltk.word_tokenize(gfile)  #tokenize to create list that I can iterate through I learned about split a bit too late
    A=[]
    for word in tokens:
        if re.match(r'^[A-Z]{2}$|^\d[A-Z]$|^[A-Z]\d$|^\d{2}$',word):#regex to match what Im searching for
            if not word in A:   #avoiding duplicates
                A.append(word)
    return A

In [ ]:
def read_edges(gfile):
    gfile=urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat').readlines()[566:674] #cut lines to what Im interested in
    returnarray=[]    #had to revisit this part and use readlines for accuracy. Hence the line above.
    for line in gfile:  #Iterate through all lines
        line=line.strip() #get rid of /n s
        
        for word in line.split(':')[1].split(';'):  #look once beyond : and iterate through ;
            limboarray=list(itertools.combinations(word.split(','),2)) #itertools to get combinations of the lists craeted by the iteration above
            #limbo array stores these values 
            for word in limboarray:   #iterate through limbo array to add these combinations to the actual array cumulatively
                if not word in returnarray: #avoiding duplicates. Wasn't necessary I guess
                    returnarray.append(word)
            
    return returnarray

In [ ]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

In [ ]:

def Search2(graph,root,array):
    
    for node in sorted(graph.neighbors(root)):   
        
        if not node in array:    # instead of indexing and coloring them each step Im using a global array to store 
            array.append(node)   # values of the visited arrays. Sorted() function ensures alphabetical order.
            Search2(graph,node,array)
            
    return  array

def Search(graph,root):   #Because search was recursive I had to use this to be able to create an initial array Q
    Q=[root]              # Q's first item is root so the above Search2 algorithm doesnt add it
    return Search2(graph,root,Q)  #Call Search2 function which does DFS


In [ ]:
ulysses = Search(G, 'OD')
print ulysses

In [ ]:
def connected_components(graph):
    returnarray2=[];ifnotarray=[]  #ifnotarray is where I will store the already visited nodes
    
    for node in sorted(graph.nodes()):  # Iterate through all nodes of the graph and run DFS
        if not node in ifnotarray:      # making sure I don't run DFS on nodes already visited
            returnarray2.append(Search(graph,node))    #add the connected components to my array
            
            for x in Search(graph,node):  #add every item visited by Search to my ifnotarray
                ifnotarray.append(x)

    return returnarray2

In [ ]:
character_interactions = connected_components(G)

In [ ]:
component_sizes = [len(c) for c in character_interactions]  
print "There are 12 connected components in the Iliad:", len(component_sizes) == 12
print "The giant component has size 542:", max(component_sizes) == 542
print "There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5